In [3]:
import glob

# HTML 파일 리스트 가져오기
html_files = glob.glob(f"../docs/blender_python_reference_4_4/**/*.html", recursive=True)

print(f"🔍 HTML 파일 개수: {len(html_files)} 개")
print(html_files[:5])  # 일부 파일 확인

🔍 HTML 파일 개수: 1760 개
['../docs/blender_python_reference_4_4\\aud.html', '../docs/blender_python_reference_4_4\\bgl.html', '../docs/blender_python_reference_4_4\\blf.html', '../docs/blender_python_reference_4_4\\bl_math.html', '../docs/blender_python_reference_4_4\\bmesh.geometry.html']


In [6]:
import os
from bs4 import BeautifulSoup

# 기준이 되는 HTML 코드 조각
trigger_code = """<div class="theme-toggle-container theme-toggle-content">"""
footer_code = "<footer>"

def clean_html(html_path):
    with open(html_path, "r", encoding="utf-8") as f:
        content = f.read()

    # 특정 코드 부분을 찾아 해당 부분 이전을 제거
    start_index = content.find(trigger_code)
    if start_index != -1:
        content = content[start_index:]  # 지정된 코드 이후만 남김

    # `<footer>` 태그를 찾아 해당 부분 이후를 제거
    end_index = content.find(footer_code)
    if end_index != -1:
        content = content[:end_index]  # 지정된 코드 이전만 남김

    # HTML을 다시 파싱하여 깔끔한 텍스트 추출
    soup = BeautifulSoup(content, "html.parser")

    # 수정된 HTML을 다시 저장
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(str(soup))

# 폴더 내 모든 HTML 파일 처리
html_folder = "../docs/blender_python_reference_4_4"  # HTML 파일이 들어있는 폴더 경로

for filename in os.listdir(html_folder):
    if filename.endswith(".html"):
        file_path = os.path.join(html_folder, filename)
        clean_html(file_path)

print("✅ HTML 파일 정제 및 저장 완료!")

✅ HTML 파일 정제 및 저장 완료!


In [7]:
from bs4 import BeautifulSoup

def clean_html(html_path):
    with open(html_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    return soup.get_text()

# 모든 HTML 파일에서 텍스트 추출
docs = [clean_html(file) for file in html_files]

print("✅ HTML 텍스트 정제 완료!")

✅ HTML 텍스트 정제 완료!


In [19]:
import textwrap

def split_text(text, max_length=1000):
    """문서를 일정 길이로 분할"""
    return textwrap.wrap(text, max_length)

# 🔥 문서를 작은 조각으로 나누기
processed_docs = []
for doc in docs:
    processed_docs.extend(split_text(doc))

print(f"📌 최종 문서 개수: {len(processed_docs)}")

📌 최종 문서 개수: 9429


In [30]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import numpy as np
import time

# 모델 이름을 문자열로 전달
model_name = "all-MiniLM-L6-v2"

# HuggingFaceEmbeddings 객체 초기화
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# 문서를 작은 배치로 나누기
batch_size = 1  # 🔽 요청당 처리할 문서 개수
vectorstore = None
total_docs = len(processed_docs)

print(f"🚀 총 {total_docs}개의 문서 임베딩 시작!")

for i in range(0, total_docs, batch_size):
    batch_docs = processed_docs[i:i+batch_size]

    # FAISS 데이터베이스 초기화 또는 추가
    if vectorstore is None:
        # FAISS 초기화 시 HuggingFaceEmbeddings 객체 전달
        vectorstore = FAISS.from_texts(batch_docs, embeddings)
    else:
        vectorstore.add_texts(batch_docs)

    # 요청 간 대기 시간 추가 (Rate Limit 방지)
    time.sleep(0.5)

    # 개별 문서 완료 표시
    print(f"✅ [{i+1}/{total_docs}] 문서 임베딩 완료!")

# 🔥 FAISS 데이터베이스 저장 추가
vectorstore.save_local("./blender_vector_db")

print("🎉 모든 문서 임베딩 및 벡터 DB 저장 완료!")


🚀 총 9429개의 문서 임베딩 시작!
✅ [1/9429] 문서 임베딩 완료!
✅ [2/9429] 문서 임베딩 완료!
✅ [3/9429] 문서 임베딩 완료!
✅ [4/9429] 문서 임베딩 완료!
✅ [5/9429] 문서 임베딩 완료!
✅ [6/9429] 문서 임베딩 완료!
✅ [7/9429] 문서 임베딩 완료!
✅ [8/9429] 문서 임베딩 완료!
✅ [9/9429] 문서 임베딩 완료!
✅ [10/9429] 문서 임베딩 완료!
✅ [11/9429] 문서 임베딩 완료!
✅ [12/9429] 문서 임베딩 완료!
✅ [13/9429] 문서 임베딩 완료!
✅ [14/9429] 문서 임베딩 완료!
✅ [15/9429] 문서 임베딩 완료!
✅ [16/9429] 문서 임베딩 완료!
✅ [17/9429] 문서 임베딩 완료!
✅ [18/9429] 문서 임베딩 완료!
✅ [19/9429] 문서 임베딩 완료!
✅ [20/9429] 문서 임베딩 완료!
✅ [21/9429] 문서 임베딩 완료!
✅ [22/9429] 문서 임베딩 완료!
✅ [23/9429] 문서 임베딩 완료!
✅ [24/9429] 문서 임베딩 완료!
✅ [25/9429] 문서 임베딩 완료!
✅ [26/9429] 문서 임베딩 완료!
✅ [27/9429] 문서 임베딩 완료!
✅ [28/9429] 문서 임베딩 완료!
✅ [29/9429] 문서 임베딩 완료!
✅ [30/9429] 문서 임베딩 완료!
✅ [31/9429] 문서 임베딩 완료!
✅ [32/9429] 문서 임베딩 완료!
✅ [33/9429] 문서 임베딩 완료!
✅ [34/9429] 문서 임베딩 완료!
✅ [35/9429] 문서 임베딩 완료!
✅ [36/9429] 문서 임베딩 완료!
✅ [37/9429] 문서 임베딩 완료!
✅ [38/9429] 문서 임베딩 완료!
✅ [39/9429] 문서 임베딩 완료!
✅ [40/9429] 문서 임베딩 완료!
✅ [41/9429] 문서 임베딩 완료!
✅ [42/9429] 문서 임베딩 완료!
✅ [43/9429] 문서 임베딩 완료